<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/dashboard_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pathlib

In [2]:
file_path = r'/content/drive/MyDrive/Dashboard_data/utils/auxiliary_tables.xlsx'

sheets_info = {
    'n_uni': {'skiprows': 2, 'cols': ['uni']},
    'n_proj': {'skiprows': 3, 'cols': ['proj', 'nome_proj']},
    'n_elem': {'skiprows': 3, 'cols': ['elem', 'nome_elem']},
    'n_vinc': {'skiprows': 3, 'cols': ['vinc', 'nome_vinc']},
    'n_rub': {'skiprows': 3, 'cols': ['rub', 'nome_rub']}
}

dfs = {
    sheet: pd.read_excel(file_path, sheet_name=sheet, skiprows=info['skiprows'], header=None, names=info['cols'])
    for sheet, info in sheets_info.items()
}

for sheet, df in dfs.items():
  if 'uni' in df.columns:
      df['nome_uni'] = df['uni'].str[5:]
      df['uni_int'] = df['uni'].str[:4].astype(int)
  if 'proj' in df.columns:
      df['proj_int'] = df['proj'].str[:4].astype(int)
  if 'elem' in df.columns:
      df['elem_int'] = df['elem'].str[:6].astype(int)
  if 'rub' in df.columns:
      df['rub_int'] = df['rub'].str[:12].astype('int64')
  if 'vinc' in df.columns:
      df['vinc_int'] = df['vinc'].str[:4].astype(int)

df_uni, df_proj, df_elem, df_vinc, df_rub = dfs.values()

In [3]:
# Generating a dataframe with paid expenses data from 2002 to 2023

exp_folder_2002_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2002_2023')

df_list = [
    pd.read_excel(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2002_2023.iterdir()
]

df_exp_2002_2023 = pd.concat(df_list, ignore_index=True)

df_exp_2002_2023 = df_exp_2002_2023[['CPTPAG', 'RESUL_PAGO', 'UNIDORC', 'PROJATIV', 'RUBRICA', 'VINCORC']].rename(columns={
    'CPTPAG':'Comp.pagto.', 'RESUL_PAGO':'Result. pago', 'UNIDORC':'Unid. Orçam.', 'PROJATIV':'Proj/Ativ',
    'RUBRICA':'Rubrica', 'VINCORC':'Vinc. Orçam.'
})

df_exp_2002_2023 = df_exp_2002_2023.dropna(subset=['Comp.pagto.', 'Result. pago'], ignore_index=True)
df_exp_2002_2023['Elemento'] = df_exp_2002_2023['Rubrica'].astype(str).str[:6].astype(int)

In [4]:
# Generating a dataframe with paid expenses data from 2024 onwards

exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

df_list = [
    pd.read_csv(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2024_onwards.iterdir()
]

df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

cols_to_convert = ['valorpago', 'restospagarnaoprocessadospagos', 'restospagarprocessadospagos']

df_exp_2024_onwards[cols_to_convert] = (
    df_exp_2024_onwards[cols_to_convert]
    .apply(lambda col: col.str.removeprefix('R$ ').str.replace(',', '.', regex=False).astype(float))
)

df_exp_2024_onwards['Result. pago'] = df_exp_2024_onwards[cols_to_convert].sum(axis=1, numeric_only=True)

df_exp_2024_onwards['mes'] = df_exp_2024_onwards['mes'].replace({0: 1, 13: 12})
df_exp_2024_onwards.rename(columns={
    'exercicio': 'YEAR', 'mes': 'MONTH', 'subacao': 'Proj/Ativ', 'unidadeorcamentaria': 'Unid. Orçam.'
    }, inplace=True)
df_exp_2024_onwards['Comp.pagto.'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))
df_exp_2024_onwards['Elemento'] = df_exp_2024_onwards['elementocompleto'].str[:8].str.replace('.', '', regex=False).astype(int)
df_exp_2024_onwards['Rubrica'] = df_exp_2024_onwards['desdobramentocompleto'].str[:12].str.replace('.', '', regex=False).astype(int)

df_exp_2024_onwards = df_exp_2024_onwards.loc[df_exp_2024_onwards['Result. pago'] != 0].sort_values('Comp.pagto.').reset_index(drop=True)
df_exp_2024_onwards['Vinc. Orçam.'] = df_exp_2024_onwards['defonterecursos'].str[:4].astype(int)
df_exp_2024_onwards = df_exp_2024_onwards[[
    'Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.',
    'deelemento', 'dedesdobramento', 'desubacao', 'deunidadeorcamentária', 'defonterecursos'
    ]]

In [5]:
relevant_columns = ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
df_exp = pd.concat([df_exp_2002_2023, df_exp_2024_onwards[relevant_columns]], ignore_index=True)

In [6]:
# def remove_line_breaks(df):
#     return df.map(lambda x: x.replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)

In [7]:
# Updating the elements dataframe

df_new_elem = df_exp_2024_onwards[['Elemento', 'deelemento']].drop_duplicates(keep='last').rename(columns={
    'Elemento': 'elem_int',
    'deelemento': 'nome_elem'
})
df_elem = pd.concat([df_elem, df_new_elem], ignore_index=True).drop_duplicates(subset='elem_int', keep='last').reset_index(drop=True)
df_elem['elem'] = df_elem['elem_int'].astype(str) + ' - ' + df_elem['nome_elem'].str.title()
# df_elem = remove_line_breaks(df_elem)

In [8]:
# Updating the rubric dataframe

df_new_rub = df_exp_2024_onwards[['Rubrica', 'dedesdobramento']].drop_duplicates(keep='last').rename(columns={
    'Rubrica': 'rub_int',
    'dedesdobramento': 'nome_rub'
})
df_rub = pd.concat([df_rub, df_new_rub], ignore_index=True).drop_duplicates(subset='rub_int', keep='last').reset_index(drop=True)
df_rub['rub'] = df_rub['rub_int'].astype(str) + ' - ' + df_rub['nome_rub'].str.title()
# df_rub = remove_line_breaks(df_rub)

In [9]:
# Updating the project dataframe

df_new_proj = df_exp_2024_onwards[['Proj/Ativ', 'desubacao']].drop_duplicates(keep='last').rename(columns={
    'Proj/Ativ': 'proj_int',
    'desubacao': 'nome_proj'
})
df_proj = pd.concat([df_proj, df_new_proj], ignore_index=True).drop_duplicates(subset='proj_int', keep='last').reset_index(drop=True)
df_proj['proj'] = df_proj['proj_int'].astype(str) + ' - ' + df_proj['nome_proj'].str.title()
# df_proj = remove_line_breaks(df_proj)

In [10]:
# Updating the budget units dataframe

df_new_uni = df_exp_2024_onwards[['Unid. Orçam.', 'deunidadeorcamentária']].drop_duplicates(keep='last').rename(columns={
    'Unid. Orçam.': 'uni_int',
    'deunidadeorcamentária': 'nome_uni'
})
df_uni = pd.concat([df_uni, df_new_uni], ignore_index=True).drop_duplicates(subset='uni_int', keep='last').reset_index(drop=True)
df_uni['uni'] = df_uni['uni_int'].astype(str) + ' - ' + df_uni['nome_uni'].str.title()
# df_uni = remove_line_breaks(df_uni)

In [11]:
# Updating the entail dataframe

df_new_vinc = df_exp_2024_onwards[['Vinc. Orçam.', 'defonterecursos']].drop_duplicates(keep='last').rename(columns={
    'Vinc. Orçam.': 'vinc_int', 'defonterecursos': 'nome_vinc'
})
df_new_vinc['nome_vinc'] = df_new_vinc['nome_vinc'].str.split('-').str.get(1).str.strip()
df_vinc = pd.concat([df_vinc, df_new_vinc], ignore_index=True).drop_duplicates(subset='vinc_int', keep='last').reset_index(drop=True)
df_vinc['vinc'] = df_vinc['vinc_int'].astype(str) + ' - ' + df_vinc['nome_vinc'].str.title()
# df_vinc = remove_line_breaks(df_vinc)

In [12]:
final_data_folder = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/final_data')

df_uni.sort_values('uni_int').to_csv(final_data_folder / 'df_uni.csv', encoding='utf-8-sig', sep=';', index=False)
df_proj.sort_values('proj_int').to_csv(final_data_folder / 'df_proj.csv', encoding='utf-8-sig', sep=';', index=False)
df_elem.sort_values('elem_int').to_csv(final_data_folder / 'df_elem.csv', encoding='utf-8-sig', sep=';', index=False)
df_rub.sort_values('rub_int').to_csv(final_data_folder / 'df_rub.csv', encoding='utf-8-sig', sep=';', index=False)
df_vinc.sort_values('vinc_int').to_csv(final_data_folder / 'df_vinc.csv', encoding='utf-8-sig', sep=';', index=False)
df_exp.sort_values('Comp.pagto.').to_csv(final_data_folder / 'df_exp.csv', encoding='utf-8-sig', sep=';', index=False)

In [13]:
df_exp_2024_onwards

,Comp.pagto.,Result. pago,Unid. Orçam.,Proj/Ativ,Elemento,Rubrica,Vinc. Orçam.,deelemento,dedesdobramento,desubacao,deunidadeorcamentária,defonterecursos
0,2024-01-01,350329.90,7001,4396,319011,31901101,6069,Vencimentos E Vantagens Fixas - Pessoal Civil,Vencimentos E Salários - Servidores Rpps,ADMINISTRAÇÃO DE PESSOAL - PREVIMPA,ADMINISTRAÇÃO PREVIMPA,6069 - TAXA DE ADMINISTRAÇÃO DO RPPS
1,2024-01-01,701267.85,7002,2744,319003,31900301,6049,Pensões,Pensões - Pessoal Civil,INATIVOS/PENSIONISTAS - SMED - R. SIMPLES,REGIME DE REPARTIÇÃO SIMPLES,6049 - PLANO FINANCEIRO DO RPPS
2,2024-01-01,8886238.27,7002,2744,319001,31900101,6049,"Aposentadorias, Reserva Remunerada E Reformas",Proventos - Pessoal Civil,INATIVOS/PENSIONISTAS - SMED - R. SIMPLES,REGIME DE REPARTIÇÃO SIMPLES,6049 - PLANO FINANCEIRO DO RPPS
3,2024-01-01,26515.72,7002,2742,319092,31909201,6049,Despesas De Exercícios Anteriores,"Aposentadorias, Reserva Remunerada E Reformas",INATIVOS/PENSIONISTAS - SMS - R. SIMPLES,REGIME DE REPARTIÇÃO SIMPLES,6049 - PLANO FINANCEIRO DO RPPS
4,2024-01-01,514.06,7002,2742,319003,31900303,6049,Pensões,13º Salário - Pessoal Civil - Pensionistas,INATIVOS/PENSIONISTAS - SMS - R. SIMPLES,REGIME DE REPARTIÇÃO SIMPLES,6049 - PLANO FINANCEIRO DO RPPS
...,...,...,...,...,...,...,...,...,...,...,...,...
1813,2025-02-01,185891.11,7001,4471,339040,33904011,6069,Serviços De Tecnologia Da Informação E Comunic...,Suporte De Infraestrutura De T.I.C.,"TECNOLOGIAS DA INFORMAÇÃO, COMUNICAÇÕES E GEOP...",ADMINISTRAÇÃO PREVIMPA,6069 - TAXA DE ADMINISTRAÇÃO DO RPPS
1814,2025-02-01,1416416.15,7001,9071,339047,33904712,1,Obrigações Tributárias E Contributivas,Pasep - Servidores,ENCARGOS ESPECIAIS - PREVIMPA,ADMINISTRAÇÃO PREVIMPA,0001 - RECURSO LIVRE ADM DIRETA - IMPOSTOS E T...
1815,2025-02-01,3599.35,7003,9075,339047,33904712,6050,Obrigações Tributárias E Contributivas,Pasep - Servidores,ENCARGOS ESPECIAIS - REGIME DE CAPITALIZAÇÃO,REGIME DE CAPITALIZAÇÃO,6050 - CAPITALIZAÇÃO
1816,2025-02-01,311.95,7001,2529,339039,33903928,6069,Outros Serviços De Terceiros - Pessoa Jurídica,Despesas Com Correios,ADMINISTRAÇÃO GERAL - PREVIMPA,ADMINISTRAÇÃO PREVIMPA,6069 - TAXA DE ADMINISTRAÇÃO DO RPPS


**Revenue**

In [ ]:
# Generating a dataframe with revenue collected from 2004 to 2017

revenue_file_2004_2017 = r'/content/drive/MyDrive/Dashboard_data/Receitas/2004_2017/2004_2017_Cubo_Antigo.xlsx'

df_rev_2004_2017 = pd.read_excel(revenue_file_2004_2017)
df_rev_2004_2017.drop(0, inplace=True)
df_rev_2004_2017.rename(columns={'Exercício': 'YEAR',
                                 'Mês': 'MONTH',
                                 'Rótulos de Linha': 'orgao',
                                 'Vínculo Cod': 'vinculo',
                                 'N6 Subalinea': 'desdobramento6',
                                 'Valor Arrecadado': 'valor_arrecadado',
                                 'Valor Orçado': 'valor_orcado'}, inplace=True)
df_rev_2004_2017['Data'] = pd.to_datetime(df_rev_2004_2017[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']] = df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']].astype('int64')
df_rev_2004_2017['vinculo'] = df_rev_2004_2017['vinculo'].astype('str').replace('400', '0400').replace('6051', '605X')\
  .replace('1', '0001').replace('605X', '6051')

In [ ]:
# Generating a dataframe with revenue collected from 2018 to 2023

rev_folder_2018_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2018_2023')

df_list = [
    pd.read_excel(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2018_2023.iterdir()
]

df_rev_2018_2023 = pd.concat(df_list, ignore_index=True)

df_rev_2018_2023.drop(columns=['valor_cancelado', 'valor_lancado', 'valor_meta', 'informacao_complementar'], inplace=True)
df_rev_2018_2023 = df_rev_2018_2023.loc[df_rev_2018_2023['orgao_raiz'] == 7000].reset_index(drop=True)
df_rev_2018_2023.rename(columns={'ano': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2018_2023['Data'] = pd.to_datetime(df_rev_2018_2023[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2018_2023['vinculo'] = df_rev_2018_2023['vinculo'].astype(str).replace({'400': '0400'})
df_rev_2018_2023['rubrica'] = df_rev_2018_2023['rubrica'].astype(str)

df_rev_2018_2023['Natureza da Receita Antiga'] = df_rev_2018_2023['rubrica'].str.pad(14, fillchar='0').str.replace(
    r'(\d{2})(\d)(\d)(\d)(\d{2})(\d)(\d)(\d{2})(\d{2})(\d+)', r'\1.\2.\3.\4.\5.\6.\7.\8.\9.\10', regex=True
) + '.' + df_rev_2018_2023['vinculo'] + '.0'

In [ ]:
# Generating a dataframe with revenue collected from 2024 onwards

rev_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2024+')

df_list = [
    pd.read_csv(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2024_onwards.iterdir()
]

df_rev_2024_onwards = pd.concat(df_list, ignore_index=True)

df_rev_2024_onwards.rename(columns={'exercicio': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2024_onwards['Data'] = pd.to_datetime(df_rev_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

In [10]:
df_exp_2002_2023.loc[df_exp_2002_2023['Comp.pagto.'].dt.year == 2023, 'Result. pago'].sum()

1850336804.21